In [1]:
import sys
import torch
import numpy as np
import pandas as ps
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn import metrics


sys.path.append("..")
device = torch.device("cuda:0")

In [2]:
from src.models import BinaryEfficientNet
from src.metrics import _alaska_weighted_auc
from src.datasets import ImagesDataset
from src.experiment import VALID_AUGMENTATIONS

/home/dmdr/anaconda3/envs/torch/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/dmdr/anaconda3/envs/torch/lib/python3.7/site-packages/nbformat/notebooknode.py:4: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working

/home/dmdr/anaconda3/envs/torch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/dmdr/anaconda3/envs/torch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/dmdr/anaconda3/envs/torch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size

In [3]:
model = BinaryEfficientNet("efficientnet-b0")
model.load_state_dict(
    torch.load("../logs/efficientnet-b0-magic-constant/checkpoints/best.pth", map_location="cpu")["model_state_dict"]
)
model = model.to(device)
model = model.eval()

Loaded pretrained weights for efficientnet-b0


In [4]:
data = ps.read_csv("../data/combined.csv")
# data = data[data["folds"] == 0]
data["images"] = "../" + data["images"]

print(data.shape)
print(data["labels"].value_counts())
data.head()

(300000, 4)
1    225000
0     75000
Name: labels, dtype: int64


,images,labels,classes,folds
0,../data/resized_data/Cover/28292.jpg,0,0,0
1,../data/resized_data/Cover/75205.jpg,0,0,0
2,../data/resized_data/Cover/75472.jpg,0,0,3
3,../data/resized_data/Cover/32819.jpg,0,0,4
4,../data/resized_data/Cover/66313.jpg,0,0,1


In [5]:
dataset = ImagesDataset(data["images"].values, transforms=VALID_AUGMENTATIONS)
dataloader = DataLoader(dataset, batch_size=64, num_workers=16)

In [6]:
preds = []
with torch.no_grad():
    for batch in tqdm(dataloader):
        batch = batch.to(device)
        out = torch.sigmoid(model(batch)).detach().cpu().numpy().flatten()
        preds.append(out)

preds = np.concatenate(preds)
print(preds.shape)

100%|██████████| 4688/4688 [37:07<00:00,  2.10it/s]

(300000,)


## score on all train data (train + validation)

**NOTE:** this score will be different from score on validation

In [7]:
labels = data["labels"].values

auc = metrics.roc_auc_score(labels, preds)
weighted_auc = _alaska_weighted_auc(labels, preds)
accuracy = (labels == (preds >= 0.5)).mean()


print(f'   Accuracy = {round(accuracy, 5)}')
print(f'        AUC = {round(auc, 5)}')
print(f'WeightedAUC = {round(weighted_auc, 5)}')

   Accuracy = 0.75463
        AUC = 0.74008
WeightedAUC = 0.81234


In [8]:
print(metrics.classification_report(labels, preds >= 0.5))

              precision    recall  f1-score   support

           0       0.60      0.06      0.10     75000
           1       0.76      0.99      0.86    225000

    accuracy                           0.75    300000
   macro avg       0.68      0.52      0.48    300000
weighted avg       0.72      0.75      0.67    300000



In [9]:
pred_classes = (preds >= 0.5)

print("Positive accuracy -", (labels == pred_classes)[labels == 1].mean())
print("Negative accuracy -", (labels == pred_classes)[labels == 0].mean())

Positive accuracy - 0.9870533333333333
Negative accuracy - 0.05736


## validation score

In [10]:
fold_labels = data[data["folds"] == 0]["labels"].values
fold_preds = preds[data["folds"] == 0]


auc = metrics.roc_auc_score(fold_labels, fold_preds)
weighted_auc = _alaska_weighted_auc(fold_labels, fold_preds)
accuracy = (fold_labels == (fold_preds >= 0.5)).mean()


print(f'   Accuracy = {round(accuracy, 5)}')
print(f'        AUC = {round(auc, 5)}')
print(f'WeightedAUC = {round(weighted_auc, 5)}')

   Accuracy = 0.75417
        AUC = 0.73491
WeightedAUC = 0.80841


In [11]:
print(metrics.classification_report(fold_labels, fold_preds >= 0.5))

              precision    recall  f1-score   support

           0       0.59      0.06      0.10     15000
           1       0.76      0.99      0.86     45000

    accuracy                           0.75     60000
   macro avg       0.67      0.52      0.48     60000
weighted avg       0.72      0.75      0.67     60000



In [12]:
pred_classes = (fold_preds >= 0.5)

print("Positive accuracy -", (fold_labels == pred_classes)[fold_labels == 1].mean())
print("Negative accuracy -", (fold_labels == pred_classes)[fold_labels == 0].mean())

Positive accuracy - 0.9872
Negative accuracy - 0.05506666666666667
